In [1]:

# Install & Import Libraries
!pip install tensorflow matplotlib opendatasets

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import opendatasets as od
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

In [2]:

#  Download Dataset from Kaggle

od.download("https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification")

# Adjust dataset path
dataset_path = "/content/garbage-classification/Garbage classification/Garbage classification"
print(os.listdir(dataset_path))  # will show folders like cardboard, glass, etc


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: layalalsultan
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification


100%|██████████| 82.0M/82.0M [00:00<00:00, 2.70GB/s]

['paper', 'glass', 'trash', 'plastic', 'cardboard', 'metal']


In [3]:

# Load & Preprocess Images

image_size = 224
data, labels = [], []

class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
class_map = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            img = img / 255.0
            data.append(img)
            labels.append(class_map[class_name])

data = np.array(data)
labels = np.array(labels)

In [4]:

#  Split Dataset
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [15]:

#  Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_datagen.fit(X_train)

In [6]:

# Mount Google Drive
drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/waste_model_v2.h5"

Mounted at /content/drive


In [11]:
# Train Model (Using Transfer Learning)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_path = "/content/drive/MyDrive/waste_model_v2.h5"

if os.path.exists(model_path):
    print(" نموذج محفوظ موجود بالفعل، سيتم تحميله")
    model = load_model(model_path)
    print(" تم تحميل النموذج بنجاح، لا حاجة لإعادة التدريب")
else:
    print("(Transfer Learningباستخدام ) لا يوجد نموذج محفوظ، سيتم البدء في تدريب جديد")

    # 1. Load the previously trained base model (MobileNetV2)
    base_model = MobileNetV2(
        input_shape=(image_size, image_size, 3),
        include_top=False,  # We do not want to include the final ImageNet classifier layer
        weights='imagenet'
    )

    # 2. Freeze the base model
    base_model.trainable = False # We don't want to re train the layers that already learned the features

    # 3. Adding a classifier "head"
    # We implemented the Keras functional API
    inputs = Input(shape=(image_size, image_size, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(6, activation='softmax')(x) # 6 classes

    # 4. Create new model
    model = Model(inputs, outputs)

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # 5. Defining when to stop
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,             # Stop after 5 epochs of no improvement
        restore_best_weights=True # Keep the best version of the model
    )
    # If the model gets "stuck", reduce the learning rate
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
    callbacks_list = [early_stop, reduce_lr]

    # 6. Train model
    history = model.fit(
        train_datagen.flow(X_train, y_train, batch_size=32),
        epochs=50,  # Increased epochs with the help of EarlyStopping to find the best one
        validation_data=(X_val, y_val),
        callbacks=callbacks_list # Use the callbacks from step 5
    )

    # 7. Save model in google drive to prevent re training every time
    model.save(model_path)
    print(" تم حفظ النموذج في Google Drive للاستخدام لاحقاً")


 نموذج محفوظ موجود بالفعل، سيتم تحميله


 تم تحميل النموذج بنجاح، لا حاجة لإعادة التدريب


In [12]:

# Evaluate Model

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f" دقة النموذج: {test_acc*100:.2f}%")


12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 957ms/step - accuracy: 0.9434 - loss: 0.2108
 دقة النموذج: 93.42%


In [13]:

#  Predict with Arabic Labels

arabic_labels = {
    'cardboard': 'كرتون',
    'glass': 'زجاج',
    'metal': 'معدن',
    'paper': 'ورق',
    'plastic': 'بلاستيك',
    'trash': 'نفايات'
}

def predict_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f" خطأ: تعذر تحميل الصورة من {img_path}")
        return
    img = cv2.resize(img, (image_size, image_size))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    class_idx = np.argmax(prediction)
    class_name = class_names[class_idx]
    print(f" الفئة المتوقعة: {arabic_labels[class_name]} ({class_name})")

# Example usage:
predict_image("/content/garbage-classification/Garbage classification/Garbage classification/glass/glass1.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step
 الفئة المتوقعة: زجاج (glass)


In [16]:
import gradio as gr

def classify_image_text_only(image):
    import cv2, numpy as np
    img = cv2.resize(image, (image_size, image_size))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    class_idx = np.argmax(prediction)
    class_name = class_names[class_idx]
    arabic_label = arabic_labels[class_name]
    return f" الفئة المتوقعة: {arabic_label}"

interface_text = gr.Interface(
    fn=classify_image_text_only,
    inputs=gr.Image(type="numpy", label=" حمّل صورة النفايات"),
    outputs=gr.Textbox(label="النتيجة بالعربية"),
    title="نظام تصنيف النفايات بالصور",
    description="اختر صورة نفايات لمعرفة نوعها (ورق، بلاستيك، زجاج، كرتون، معدن، نفايات)."
)

interface_text.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3a3219790ec5faad8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
